In [1]:
import numpy as np
from tqdm import tqdm
from utils import read_data, read_query, read_output, knn, distances, split_normal_distribution, _recall, recall

In [2]:
#data_path = '../data/dummy-data.bin'
#query_path = '../data/dummy-queries.bin'


# output_path_1 = '../data/dummy-output-seqscan.bin'
#output_path_2 = '../data/dummy-output-seqscanrange.bin'
# output_path_2 = '../data/dummy-output-seqscanrangemt.bin'

In [3]:
data_path = '../data/contest-data-release-1m.bin'
query_path = '../data/contest-queries-release-1m.bin'

output_path_1 = '../data/relsmall-normal.bin'
output_path_2 = '../data/relsmall-normal-range.bin'

In [4]:
# read data, query and output1, output2
data = read_data(data_path)
query = read_query(query_path) 
output1 = read_output(output_path_1, query.shape[0])
output2 = read_output(output_path_2, query.shape[0])
# calculate distances of outputs

print("="*10)
print("data shape", data.shape)
print("query shape", query.shape)
print("="*10)
print("output1 shape", output1.shape)
print("output2 shape", output2.shape)
print("="*4,"KNN","="*4)


# True knn
output_true, dist_true = knn(data, query, 100)
#dist_output1 = distances(output1, dist)
#dist_output2 = distances(output2, dist)

print("="*10)
print("output_true shape", output_true.shape)
print("dist_true shape", dist_true.shape)
print("="*10)




data shape (1000000, 102)
query shape (10000, 104)
output1 shape (10000, 100)
output2 shape (10000, 100)
==== KNN ====


  9%|▉         | 909/10000 [16:02<2:40:21,  1.06s/it]


KeyboardInterrupt: 

testing _recall_

In [22]:
rec, _  = recall(output_true, output1, dist_true, dist_output1, verbose=False)
rec 

1.0

In [23]:
rec, _ = recall(output_true, output2, dist_true, dist_output2, verbose=False)
rec 

1.0

### generate breakpoints

In [2]:
split_normal_distribution(2, verbose=True)

Breakpoints for 2 equal parts:
Breakpoint 1: z-score = 0.0000


array([0.])

In [3]:
split_normal_distribution(3, verbose=True)

Breakpoints for 3 equal parts:
Breakpoint 1: z-score = -0.4307
Breakpoint 2: z-score = 0.4307


array([-0.4307273,  0.4307273])

In [4]:
split_normal_distribution(8, verbose=True)

Breakpoints for 8 equal parts:
Breakpoint 1: z-score = -1.1503
Breakpoint 2: z-score = -0.6745
Breakpoint 3: z-score = -0.3186
Breakpoint 4: z-score = 0.0000
Breakpoint 5: z-score = 0.3186
Breakpoint 6: z-score = 0.6745
Breakpoint 7: z-score = 1.1503


array([-1.15034938, -0.67448975, -0.31863936,  0.        ,  0.31863936,
        0.67448975,  1.15034938])

In [10]:
n_breaks = [2, 3, 8, 16, 32, 64, 128, 256, 512]

# open file
f = open("breakpoints.cpp", "w")
for n_breaks in tqdm(n_breaks):
    breaks = split_normal_distribution(n_breaks)
    # write to file
    f.write("const float BREAK_POINTS_" + str(n_breaks) + "[] = {")
    for i in range(len(breaks)):
        f.write(str(breaks[i]) + ",")
        
    f.write("};\n")
f.close()

100%|██████████| 9/9 [00:00<00:00, 2525.51it/s]


### testing __'_recall'__ : simple recall between two 1D vectors 

In [11]:
a = np.array([5,10,20,30,np.inf])
a_idx = np.array([0,1,2,3,5])

b = np.array([10,20,30,40,40])
b_idx = np.array([1,2,3,4,4])

# replace inf with max value and return the index of the max value
a[a == np.inf] = np.max(a[np.isfinite(a)])
a.argmax()


_recall(a_idx,b_idx,a,b)


0.75

In [12]:
a = np.array([5,10,20,30,40])
a_idx = np.array([0,1,2,3,5])

b = np.array([10,20,30,40,40])
b_idx = np.array([1,2,3,4,4])

# replace inf with max value and return the index of the max value
a[a == np.inf] = np.max(a[np.isfinite(a)])
a.argmax()


_recall(a_idx,b_idx,a,b)

0.8

In [13]:
a = np.array([5,10,20,30,35])
a_idx = np.array([0,1,2,3,5])

b = np.array([5,10,20,30,40])
b_idx = np.array([0,1,2,3,6])

# replace inf with max value and return the index of the max value
a[a == np.inf] = np.max(a[np.isfinite(a)])
a.argmax()


_recall(a_idx,b_idx,a,b)

0.8